In [1]:
import pandas as pd

In [2]:
debt = pd.read_csv('debt.csv')

In [3]:
debt.rename(columns={'2019 [YR2019]': '2019', '2020 [YR2020]': '2020', '2021 [YR2021]': '2021', '2022 [YR2022]': '2022', '2023 [YR2023]': '2023'}, inplace=True)

In [4]:
debt.dropna(inplace=True)

In [5]:
debt['Country Name'] = debt['Country Name'].str.strip()
debt['Counterpart-Area Name'] = debt['Counterpart-Area Name'].str.strip()

In [6]:
debt = debt[(debt['2019'] != '..') & (debt['2020'] != '..') & (debt['2021'] != '..') & (debt['2022'] != '..') & (debt['2023'] != '..')]

In [7]:
debt = debt.melt(id_vars=['Country Name', 'Country Code', 'Counterpart-Area Name'], value_vars=['2019', '2020', '2021', '2022', '2023'], var_name='Year', value_name='Debt')

In [8]:
debt['Debt'] = debt['Debt'].astype(float)

In [10]:
pd.DataFrame(debt.groupby('Counterpart-Area Name')['Debt'].sum().sort_values(ascending=False))

,Debt
Counterpart-Area Name,
World,4.316449e+11
Other Multiple Lenders,1.671310e+11
China,6.572703e+10
Bondholders,4.959060e+10
World Bank-IDA,2.081378e+10
...,...
Grenada,4.588891e+05
St. Lucia,3.034261e+05
International Bank for Economic Cooperation (IBEC),1.855000e+05


In [91]:
def save_top(df, n=10):
    df = df.copy()
    # only save top 100 lenders
    top_lenders = df.reset_index().groupby('Counterpart-Area Name')['Debt'].sum().sort_values(ascending=False).nlargest(n).index
    df = df[(df.index.get_level_values(0).isin(top_lenders)) & (~df.index.get_level_values(0).isin(['World', 'Other Multiple Lenders']))]
    print(df)
    return df

In [92]:
grouped_2019 = debt[debt['Year'] == "2019"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()
grouped_2020 = debt[debt['Year'] == "2020"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()
grouped_2021 = debt[debt['Year'] == "2021"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()
grouped_2022 = debt[debt['Year'] == "2022"].groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()

grouped_2019 = save_top(grouped_2019)
grouped_2020 = save_top(grouped_2020)
grouped_2021 = save_top(grouped_2021)
grouped_2022 = save_top(grouped_2022)

import json

def save_json(grouped, year):
    levels = len(grouped.index.levels)
    dicts = [{} for i in range(levels)]
    last_index = None

    for index,value in grouped.itertuples():

        if not last_index:
            last_index = index

        for (ii,(i,j)) in enumerate(zip(index, last_index)):
            if not i == j:
                ii = levels - ii -1
                dicts[:ii] =  [{} for _ in dicts[:ii]]
                break

        for i, key in enumerate(reversed(index)):
            dicts[i][key] = value
            value = dicts[i]

        last_index = index


    result = dicts[-1]
    # result = json.dumps(dicts[-1])

    debt_restructured = {'name': 'debt', 'children': []}

    for i in result:
        obj = {'name': i, 'children': []}
        for j in result[i]:
            obj['children'].append({'name': j, 'value': result[i][j]})
        debt_restructured['children'].append(obj)
        
    with open(f'debt_{year}.json', 'w') as f:
        f.write(json.dumps(debt_restructured, indent=4))
    
    with open(f'debt_{year}.json', 'r+') as file:
        file.seek(0)
        file.write('var data = ')
        file.write(json.dumps(debt_restructured, indent=4))
        file.write(';')

save_json(grouped_2019, 2019)
save_json(grouped_2020, 2020)
save_json(grouped_2021, 2021)
save_json(grouped_2022, 2022)

                                           Debt
Counterpart-Area Name Country Name             
Asian Dev. Bank       Afghanistan    14275000.0
                      Bangladesh    656569000.0
                      Bhutan         14238000.0
                      Cambodia       62968000.0
                      Fiji           13357000.0
...                                         ...
World Bank-IDA        Uganda         58350000.0
                      Uzbekistan     39709000.0
                      Vanuatu         1032000.0
                      Yemen, Rep.    83913000.0
                      Zambia         18857000.0

[294 rows x 1 columns]
                                           Debt
Counterpart-Area Name Country Name             
Asian Dev. Bank       Afghanistan    14273000.0
                      Bangladesh    679578000.0
                      Bhutan         15528000.0
                      Cambodia       69745000.0
                      Fiji           13774000.0
...             

In [70]:
grouped_2020.index.get_level_values(0)

Index(['Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders',
       'Other Multiple Lenders', 'Other Multiple Lenders', 'World', 'World',
       'World', 'World', 'World', 'Wor

In [49]:
grouped_2020.reset_index().groupby('Counterpart-Area Name')['Debt'].sum().sort_values(ascending=False).nlargest(100).index

Index(['World', 'Other Multiple Lenders', 'China', 'Bondholders',
       'United Kingdom', 'Asian Dev. Bank', 'International Monetary Fund',
       'World Bank-IDA', 'Islamic Dev. Bank', 'France', 'United Arab Emirates',
       'United States', 'Eastern & Southern African Trade & Dev. Bank (TDB)',
       'African Dev. Bank', 'Japan', 'Spain', 'Belgium', 'Multiple Lenders',
       'Australia', 'Central American Bank for Econ. Integration (BCIE)',
       'World Bank-IBRD', 'Germany, Fed.Rep. Of', 'Inter-American Dev. Bank',
       'Portugal', 'Russian Federation',
       'West African Development Bank - BOAD', 'South Africa', 'Switzerland',
       'Korea, Republic Of', 'Netherlands', 'African Export-Import Bank',
       'Ireland', 'Israel', 'Egypt', 'India', 'Austria', 'Brazil', 'Italy',
       'European Investment Bank', 'Turkey', 'Canada', 'Thailand',
       'International Fund for Agricultural Dev. (IFAD)',
       'Caribbean Dev. Bank', 'OPEC Fund for International Dev.',
       'Arab

In [16]:
grouped_2019
# only keep largest lenders
# grouped_2019 = grouped_2019.groupby(level=0).apply(lambda x: x.nlargest(5, 'Debt'))
grouped_2020 = grouped_2020.groupby(level=0).apply(lambda x: x.nlargest(5, 'Debt'))

In [33]:
# grouped_2020.reset_index(level=0, inplace=True)
grouped_2020.drop(columns=['Counterpart-Area Name'], inplace=True)

In [38]:
grouped_2020.index

MultiIndex([(              'African Dev. Bank',          'Nigeria'),
            (              'African Dev. Bank',           'Angola'),
            (              'African Dev. Bank',          'Somalia'),
            (              'African Dev. Bank', 'Congo, Dem. Rep.'),
            (              'African Dev. Bank',            'Kenya'),
            (     'African Export-Import Bank',      'Congo, Rep.'),
            (     'African Export-Import Bank',    'Cote d'Ivoire'),
            (     'African Export-Import Bank',          'Senegal'),
            (                         'Angola',           'Guinea'),
            ('Arab African International Bank',       'Madagascar'),
            ...
            (                'World Bank-IBRD',         'Pakistan'),
            (                'World Bank-IBRD',       'Uzbekistan'),
            (                'World Bank-IBRD',           'Angola'),
            (                'World Bank-IBRD',           'Kosovo'),
            (     

In [12]:
dicts[-1]

NameError: name 'dicts' is not defined

In [17]:
debt.to_csv('debt_clean.csv', index=False)

In [13]:
grouped_2019

,,Debt
Counterpart-Area Name,Country Name,
